In [1]:
#Cluster generated poses
#Inputs:
    #dock/poses/tropb_2_prot
    #dock/scores/tropb_2_prot.csv
#Outputs:
    #dock/cluster/tropb_2_prot.xlsx
    
import os
import sys
import re
import pandas as pd
import numpy as np
os.chdir('/home/azamh/demo/seq_struct_func/dock')

In [2]:
#Arguments
protein = 'tropb'
ligand = '2'
dockdir = f'poses/{protein}_{ligand}_prot'
energy_df = pd.read_csv(f'scores/{protein}_{ligand}_prot.csv', header = 0, index_col = 0)

In [3]:
#Run MMTSB toolkit cluster.pl
def run_clusterpl(protein, ligand, dockdir, radius = 1):
    cmd = f'cluster.pl -kclust -nolsqfit -radius {radius} -selmode heavy {dockdir}/{protein}_{ligand}_*.pdb'
    cluster_out = os.popen(cmd).read()
    return cluster_out

cluster_out = run_clusterpl(protein, ligand, dockdir)
print(cluster_out)

/export/apps/CentOS7/mmtsb/src/feig/toolset//bin/kclust -pdb -centroid -cdist -radius 1 -iterate -maxerr 0.01 -mode rmsd -heavy


# cluster file
# automatically generated on: Sun Dec 11 21:56:52 2022
# mode: rmsd, filetype: pdb, lsqfit: 0, selmode: heavy
@cluster t has 500 elements, 108 subclusters
1 poses/tropb_2_prot/tropb_2_100.pdb
2 poses/tropb_2_prot/tropb_2_101.pdb
3 poses/tropb_2_prot/tropb_2_102.pdb
4 poses/tropb_2_prot/tropb_2_103.pdb
5 poses/tropb_2_prot/tropb_2_104.pdb
6 poses/tropb_2_prot/tropb_2_105.pdb
7 poses/tropb_2_prot/tropb_2_106.pdb
8 poses/tropb_2_prot/tropb_2_107.pdb
9 poses/tropb_2_prot/tropb_2_108.pdb
10 poses/tropb_2_prot/tropb_2_109.pdb
11 poses/tropb_2_prot/tropb_2_10.pdb
12 poses/tropb_2_prot/tropb_2_110.pdb
13 poses/tropb_2_prot/tropb_2_111.pdb
14 poses/tropb_2_prot/tropb_2_112.pdb
15 poses/tropb_2_prot/tropb_2_113.pdb
16 poses/tropb_2_prot/tropb_2_114.pdb
17 poses/tropb_2_prot/tropb_2_115.pdb
18 poses/tropb_2_prot/tropb_2_116.pdb
19 poses/tropb_2_prot/tropb_2_117.pdb
20 poses/tropb_2_prot/tropb_2_118.pdb
21 poses/tropb_2_prot/tropb_2_119.pdb
22 poses/tropb_2_prot/tropb_2_11.pdb
23 po

In [4]:
#Read in cluster output and return list of clusters
def read_clusters(protein, ligand, dockdir, cluster_out):
    clusters = []
    cluster = []
    start = False
    
    for line in cluster_out.splitlines():
        if 'cluster t.' in line:
            if cluster:
                clusters.append(cluster)
            cluster = []
            start = True
        elif start:
            index = re.findall('{}/{}_{}_(\S+).pdb'.format(dockdir, protein, ligand), line)[0]
            cluster.append(index)
    if cluster:
        clusters.append(cluster)
    
    #sort clusters by size: largest -> smallest
    clusters.sort(key = len, reverse=True)  
    
    return clusters

clusters = read_clusters(protein, ligand, dockdir, cluster_out)

In [5]:
energy_df

,energy,deltae,grms,bonds,angles,ureyb,dihedrals,impropers,vdwaals,elec,hbonds,asp,user
pose,,,,,,,,,,,,,
initial,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,12.998569,-12.998569,-0.623083,0.076540,13.731054,0.332619,37.460163,0.005922,11.811688,-50.419416,0.0,0.0,0.0
2,80.810311,-67.811742,1.252078,2.339323,21.674087,1.023282,41.352234,3.706799,59.088113,-48.373527,0.0,0.0,0.0
3,55.750213,25.060098,0.020630,0.858381,20.309174,0.415126,53.555397,0.543544,15.235303,-35.166713,0.0,0.0,0.0
4,25.382261,30.367952,0.013010,-0.142033,6.428183,0.312646,26.919424,0.031123,18.869256,-27.036338,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,8.824033,43.957035,8.859730,4.043020,12.898402,0.628022,14.446895,0.089438,20.860686,-44.142431,0.0,0.0,0.0
497,37.505195,-28.681161,-0.373102,1.263480,6.855261,0.524656,32.424195,0.064691,21.857716,-25.484804,0.0,0.0,0.0
498,50.500944,-12.995749,1.724160,1.437791,20.460116,1.020070,37.312709,0.055322,37.527778,-47.312841,0.0,0.0,0.0


In [6]:
#Calculate stats for each cluster
def cluster_stats(cluster, energy_df):
    cluster_dict = dict()
    cluster_dict['cluster'] = cluster
    cluster_dict['size'] = len(cluster)
    energies = []
    min_index = -1
    min_ener = ''
    for index in cluster:
        index_energy = energy_df['energy'][index]
        energies.append(float(index_energy))
        if not min_ener or index_energy < min_ener:
            min_ener = round(index_energy,2)
            min_index = index
    cluster_dict['min_ener'] = min_ener
    cluster_dict['min_index'] = min_index
    average = round(np.average(energies),2)
    std = round(np.std(energies),2)
    cluster_dict['average energy'] = average
    cluster_dict['std energy'] = std
    return cluster_dict
cluster_dicts = [cluster_stats(cluster, energy_df) for cluster in clusters]

In [7]:
cluster_df = pd.DataFrame.from_records(cluster_dicts)
cluster_df = cluster_df.sort_values(by=['min_ener'])
cluster_df = cluster_df.reset_index(drop=True)
cluster_df.to_excel(f'cluster/{protein}_{ligand}_prot.xlsx')
cluster_df

,cluster,size,min_ener,min_index,average energy,std energy
0,"[105, 135, 147, 162, 179, 199, 224, 319]",8,-50.03,224,-28.72,15.96
1,"[108, 115, 185, 213, 233, 242, 253, 263, 270, ...",17,-46.84,185,-20.94,17.58
2,[145],1,-42.09,145,-42.09,0.00
3,"[122, 128, 136, 204, 211, 261, 282, 283, 309, ...",11,-40.18,211,15.15,30.23
4,"[194, 241, 438, 458, 471, 63]",6,-38.91,438,-33.32,3.23
...,...,...,...,...,...,...
103,[245],1,1866.75,245,1866.75,0.00
104,[287],1,3351.10,287,3351.10,0.00
105,"[156, 215]",2,3791.10,156,3799.87,8.77
106,[146],1,3817.38,146,3817.38,0.00
